In [1]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os.path
from keras.utils import np_utils
import tensorflow as tf
from pandas import Series
import time
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
import random
import os


checkpointer = ModelCheckpoint(
        filepath='/home/cis/Desktop/LStm Dense Trajectories/data/checkpoints/' + 'dt'+ \
            '.{epoch:03d}-{val_loss:.3f}.hdf5',
        verbose=1,
        save_best_only=True)

# Helper: TensorBoard
tb = TensorBoard(log_dir='/home/cis/Desktop/LStm Dense Trajectories/data/logs')

# Helper: Stop when we stop learning.
early_stopper = EarlyStopping(patience=10)

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger('/home/cis/Desktop/LStm Dense Trajectories/data/logs/'+ '-' + 'training-dt' + \
        str(timestamp) + '.log')


filepath = "/home/cis/Desktop/LStm Dense Trajectories/training.txt"
training_data_list = pd.read_csv(filepath, sep=" ", header=None)


filepath_test = "/home/cis/Desktop/LStm Dense Trajectories/testingdata.txt"
testing_data_list = pd.read_csv(filepath_test, sep=" ", header=None)

test_train_list = pd.concat([training_data_list, testing_data_list])

classes = ['boxing' , 'handclapping' , 'handwaving' , 'jogging' , 'running' , 'walking']

Using TensorFlow backend.


In [2]:
## Get label from file name and convert to categorical
    
def make_label_data(file_name, classes):
    label = file_name.split("_")
    label = label[1]
    label_encoded = classes.index(label)
    get_label = np_utils.to_categorical (label_encoded, len(classes))       
    get_label  = get_label[0]
    return get_label

## Get the size of the Dense Trajectories output in the Dataset
def get_size(alldata):
    seq_size = []
    for i in range(0, len(alldata)):
        filename = alldata[i][0]
        #print filename
        filepath_file = os.path.join("/home/cis/Desktop/LStm Dense Trajectories/Dense Trajectories" , filename)
        x= pd.read_csv(filepath_file, sep = "\t", header =None)
        size = len(x)
        seq_size.append(size)
    return seq_size


def get_min_value(test_train_list):
    print "Getting Sequence size"
    seq_size = get_size(test_train_list)
    print "Getting Minimum value"
    minvalue = seq_size[0]
    
    for i in range(0, len(seq_size)):
        if seq_size[i] < minvalue:
            minvalue = seq_size[i]
        else:
            minvalue = minvalue
    return minvalue

In [3]:
## Get the training data and labels for the file

def load_training_data(training_data_list, classes, minvalue):
    X, Y = [],[]
    for i in range(0, len(training_data_list)):
        filename = training_data_list.iloc[i][0]
        #print filename
        filepath_file = os.path.join("/home/cis/Desktop/LStm Dense Trajectories/Dense Trajectories" , filename)
        x= pd.read_csv(filepath_file, sep = "\t", header =None)
        x = x.iloc[:,10:436]
        x_sample = x.iloc[random.sample(x.index, minvalue)] 
        x_sample.sort_index(inplace=True)
        x = x_sample.values
        label = make_label_data(filename, classes)
        X.append(x)
        Y.append(label)
        ## get label
    return np.array(X), np.array(Y)    

## Load the testing data and labels

def load_testing_data(testing_data_list, classes, minvalue):
    X, Y = [],[]
    for i in range(0, len(testing_data_list)):
        filename = testing_data_list.iloc[i][0]
        #print filename
        filepath_file = os.path.join("/home/cis/Desktop/LStm Dense Trajectories/Dense Trajectories" , filename)
        x= pd.read_csv(filepath_file, sep = "\t", header =None)
        x = x.iloc[:,10:436]
        x_sample = x.iloc[random.sample(x.index, minvalue)] 
        x_sample.sort_index(inplace=True)
        x = x_sample.values
        label = make_label_data(filename, classes)
        X.append(x)
        Y.append(label)
        ## get label
    return np.array(X), np.array(Y)    

In [4]:
## Get the minimmum length of Dense Trajectoreis output in whole data set
minvalue = get_min_value(test_train_list.values)

Getting Sequence size
Getting Minimum value


In [5]:
## Get the testing Data
X_train, Y_train = load_training_data(training_data_list, classes, minvalue)

/home/cis/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
## Get the Testing Data
X_test , Y_test = load_testing_data(testing_data_list, classes, minvalue)

/home/cis/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
## Train the model
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

optimizer = Adam(lr=1e-6)
model.compile(loss='categorical_crossentropy', optimizer= optimizer,
                           metrics = ['accuracy'] )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 762, 200)          501600    
_________________________________________________________________
flatten_1 (Flatten)          (None, 152400)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               15240100  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 606       
Total params: 15,742,306
Trainable params: 15,742,306
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_data=(X_test, Y_test), verbose=1, callbacks=[checkpointer, tb, early_stopper, csv_logger])

Train on 383 samples, validate on 216 samples
Epoch 1/100
383/383 [==============================] - 23s - loss: 1.7945 - acc: 0.1775 - val_loss: 1.7684 - val_acc: 0.2870
Epoch 2/100
383/383 [==============================] - 19s - loss: 1.7670 - acc: 0.2350 - val_loss: 1.7471 - val_acc: 0.2870
Epoch 3/100
383/383 [==============================] - 20s - loss: 1.7518 - acc: 0.2245 - val_loss: 1.7271 - val_acc: 0.3704
Epoch 4/100
383/383 [==============================] - 19s - loss: 1.7255 - acc: 0.2742 - val_loss: 1.7066 - val_acc: 0.4583
Epoch 5/100
383/383 [==============================] - 20s - loss: 1.6986 - acc: 0.3420 - val_loss: 1.6887 - val_acc: 0.4907
Epoch 6/100
383/383 [==============================] - 19s - loss: 1.6755 - acc: 0.3629 - val_loss: 1.6704 - val_acc: 0.4954
Epoch 7/100
383/383 [==============================] - 19s - loss: 1.6545 - acc: 0.4047 - val_loss: 1.6533 - val_acc: 0.5556
Epoch 8/100
383/383 [==============================] - 19s - loss: 1.6348 - acc

383/383 [==============================] - 19s - loss: 1.4111 - acc: 0.5326 - val_loss: 1.4165 - val_acc: 0.6250
Epoch 25/100
383/383 [==============================] - 19s - loss: 1.3984 - acc: 0.5065 - val_loss: 1.4057 - val_acc: 0.6389
Epoch 26/100
383/383 [==============================] - 19s - loss: 1.3922 - acc: 0.5248 - val_loss: 1.3949 - val_acc: 0.6944
Epoch 27/100
383/383 [==============================] - 20s - loss: 1.3589 - acc: 0.5509 - val_loss: 1.3857 - val_acc: 0.6481
Epoch 28/100
383/383 [==============================] - 20s - loss: 1.3650 - acc: 0.5326 - val_loss: 1.3745 - val_acc: 0.6852
Epoch 29/100
383/383 [==============================] - 19s - loss: 1.3673 - acc: 0.5222 - val_loss: 1.3636 - val_acc: 0.6620
Epoch 30/100
383/383 [==============================] - 20s - loss: 1.3580 - acc: 0.5144 - val_loss: 1.3539 - val_acc: 0.6667
Epoch 31/100
383/383 [==============================] - 19s - loss: 1.3468 - acc: 0.5405 - val_loss: 1.3444 - val_acc: 0.6991
Epoch

383/383 [==============================] - 19s - loss: 1.1824 - acc: 0.6319 - val_loss: 1.2136 - val_acc: 0.7130
Epoch 48/100
383/383 [==============================] - 20s - loss: 1.1803 - acc: 0.6110 - val_loss: 1.2066 - val_acc: 0.7222
Epoch 49/100
383/383 [==============================] - 20s - loss: 1.2014 - acc: 0.5849 - val_loss: 1.1990 - val_acc: 0.7222
Epoch 50/100
383/383 [==============================] - 20s - loss: 1.1633 - acc: 0.5927 - val_loss: 1.1918 - val_acc: 0.7130
Epoch 51/100
383/383 [==============================] - 19s - loss: 1.1719 - acc: 0.6005 - val_loss: 1.1854 - val_acc: 0.7222
Epoch 52/100
383/383 [==============================] - 19s - loss: 1.1708 - acc: 0.6110 - val_loss: 1.1785 - val_acc: 0.7407
Epoch 53/100
383/383 [==============================] - 19s - loss: 1.1499 - acc: 0.5953 - val_loss: 1.1727 - val_acc: 0.7130
Epoch 54/100
383/383 [==============================] - 19s - loss: 1.1274 - acc: 0.6188 - val_loss: 1.1661 - val_acc: 0.6991
Epoch

383/383 [==============================] - 19s - loss: 1.0327 - acc: 0.6527 - val_loss: 1.0739 - val_acc: 0.7222
Epoch 71/100
383/383 [==============================] - 19s - loss: 1.0261 - acc: 0.6580 - val_loss: 1.0699 - val_acc: 0.7176
Epoch 72/100
383/383 [==============================] - 20s - loss: 1.0500 - acc: 0.6136 - val_loss: 1.0630 - val_acc: 0.7269
Epoch 73/100
383/383 [==============================] - 19s - loss: 1.0378 - acc: 0.6580 - val_loss: 1.0570 - val_acc: 0.7361
Epoch 74/100
383/383 [==============================] - 19s - loss: 0.9920 - acc: 0.6841 - val_loss: 1.0519 - val_acc: 0.7500
Epoch 75/100
383/383 [==============================] - 19s - loss: 1.0358 - acc: 0.6371 - val_loss: 1.0483 - val_acc: 0.7315
Epoch 76/100
383/383 [==============================] - 19s - loss: 0.9907 - acc: 0.7023 - val_loss: 1.0446 - val_acc: 0.7361
Epoch 77/100
383/383 [==============================] - 19s - loss: 1.0293 - acc: 0.6266 - val_loss: 1.0401 - val_acc: 0.7361
Epoch

383/383 [==============================] - 19s - loss: 0.9239 - acc: 0.6919 - val_loss: 0.9722 - val_acc: 0.7546
Epoch 94/100
383/383 [==============================] - 20s - loss: 0.9175 - acc: 0.6945 - val_loss: 0.9699 - val_acc: 0.7500
Epoch 95/100
383/383 [==============================] - 19s - loss: 0.9224 - acc: 0.7050 - val_loss: 0.9673 - val_acc: 0.7361
Epoch 96/100
383/383 [==============================] - 19s - loss: 0.9329 - acc: 0.7023 - val_loss: 0.9606 - val_acc: 0.7593
Epoch 97/100
383/383 [==============================] - 20s - loss: 0.8901 - acc: 0.7363 - val_loss: 0.9555 - val_acc: 0.7500
Epoch 98/100
383/383 [==============================] - 20s - loss: 0.9001 - acc: 0.6919 - val_loss: 0.9523 - val_acc: 0.7500
Epoch 99/100
383/383 [==============================] - 19s - loss: 0.8857 - acc: 0.7206 - val_loss: 0.9482 - val_acc: 0.7500
Epoch 100/100
383/383 [==============================] - 19s - loss: 0.8333 - acc: 0.7493 - val_loss: 0.9441 - val_acc: 0.7500
